In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F4'
CRYPTO_TARGET = 'TINYAES128C'
SS_VER = 'SS_VER_2_1'

In [ ]:
SAVE_REF = False

In [ ]:
passed = True
ERR_MSG=""

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3

In [ ]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = '../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex'.format(PLATFORM)
cw.program_target(scope, prog, fw_path)

In [ ]:
scope.clock.adc_src = "clkgen_x1"

In [ ]:
scope.adc.stream_mode = True

In [ ]:
scope.adc.basic_mode = "rising_edge"

In [ ]:
scope.adc.samples=1000000

In [ ]:
ktp = cw.ktp.Basic()
key, text = ktp.next()
ret = cw.capture_trace(scope, target, text, key)

In [ ]:
assert len(ret.wave) == 1000000, "Incorrect wave length {}".format(ret.wave)

In [ ]:
import numpy as np
if (SAVE_REF):
    np.save("pro_stream.npy", ret.wave)

In [ ]:
import numpy as np
ref_wave = np.load("pro_stream.npy", allow_pickle=True)

In [ ]:
cw.plot(ret.wave)

In [ ]:
sad = np.sum(np.abs(ref_wave - ret.wave))
#assert(sad < 5000), "Bad stream (must be <5000), SAD = {}".format(sad)

In [ ]:
SAD_point = int(120)
scope.adc.stream_mode = False

In [ ]:
scope.SAD.reference = ret.wave[SAD_point:SAD_point+128]

In [ ]:
scope.SAD.threshold = 2000

In [ ]:
scope.SAD.start()

In [ ]:
scope.trigger.module = "SAD"
scope.adc.basic_mode = "rising_edge"

In [ ]:
scope.adc.samples=5000

In [ ]:
from tqdm import tnrange
project = cw.create_project("projects/Tutorial_B5", overwrite = True)
for i in tnrange(100, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)

In [ ]:
cw.plot(project.waves[1])

In [ ]:
import chipwhisperer.analyzer as cwa
leak_model = cwa.leakage_models.sbox_output
attack = cwa.cpa(project, leak_model)

In [ ]:
results = attack.run(None, 100)

In [ ]:
print(results)

In [ ]:
print(results.find_key(), "\n", list(project.keys[0]))

In [ ]:
assert results.find_key() == list(project.keys[0]), "Failed to break key!\nGot: {}\nExp: {}".format(results.find_key(), list(project.keys[0]))